In [ ]:
!unzip '/content/drive/MyDrive/noren.zip'
!unzip '/content/drive/MyDrive/test.zip'

In [ ]:
!unzip '/content/drive/MyDrive/8a95c26e1bab11eb.zip'

In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, RMSprop 

In [14]:
train_data = pd.read_csv('/content/dataset/train.csv')
train_data['img'] = train_data['image_id'] + ".jpg"
Y = train_data[['breed']]

kf = KFold(n_splits = 5)          
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

train_data

,image_id,breed,img
0,a8b3ad1dde,nerodia-erythrogaster,a8b3ad1dde.jpg
1,8b492b973d,pantherophis-vulpinus,8b492b973d.jpg
2,929b99ea92,thamnophis-sirtalis,929b99ea92.jpg
3,bbac7385e2,pantherophis-obsoletus,bbac7385e2.jpg
4,ef776b1488,agkistrodon-contortrix,ef776b1488.jpg
...,...,...,...
5503,5f0c970adf,lampropeltis-triangulum,5f0c970adf.jpg
5504,f6a4287fac,pantherophis-alleghaniensis,f6a4287fac.jpg
5505,d3c10e63e1,thamnophis-sirtalis,d3c10e63e1.jpg
5506,61ef50879c,pituophis-catenifer,61ef50879c.jpg


In [5]:
idg = ImageDataGenerator(rescale=1./255, 
                         rotation_range=45,
                         width_shift_range=0.3,
                         height_shift_range=0.3,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [16]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

image_dir = '/content/dataset/train'
fold_var = 1
batch_size= 248

for train_index, val_index in skf.split(np.zeros(5508),Y):
  
  training_data = train_data.iloc[train_index]
  validation_data = train_data.iloc[val_index]
  
  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir, 
                                                 x_col = "img", y_col = "breed", 
                                                 class_mode = "categorical", shuffle = True,
                                                 target_size=(224, 224), batch_size=batch_size)
  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir, 
                                                 x_col = "img", y_col = "breed", 
                                                 class_mode = "categorical", shuffle = True,     
                                                 target_size=(224, 224),batch_size=batch_size)
 
 	# CREATE NEW MODEL
  from keras.applications import MobileNet

  # MobileNet was designed to work on 224 x 224 pixel input images sizes
  img_rows, img_cols = 224, 224 

  # Re-loads the MobileNet model without the top or FC layers
  MobileNet = MobileNet(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_rows, img_cols, 3))

  # Here we freeze the last 4 layers 
  # Layers are set to trainable as True by default
  for layer in MobileNet.layers:
      layer.trainable = False
  
  def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model
  
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
  from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
  from keras.layers.normalization import BatchNormalization
  from keras.models import Model

  # Set our class number to 35 snake breed
  num_classes = 35
  FC_Head = addTopModelMobileNet(MobileNet, num_classes)
  model = Model(inputs = MobileNet.input, outputs = FC_Head)
  
  # COMPILE NEW MODEL
  # We use a very small learning rate 
  model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001), metrics = ['accuracy'])
 	# CREATE CALLBACKS
  checkpoint = tf.keras.callbacks.ModelCheckpoint(get_model_name(fold_var),
                                                  monitor='val_accuracy', verbose=1,
                                                  save_best_only=True, mode='max')
 
  earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                               min_delta = 0,
                                               patience = 5,
                                               verbose = 1,
                                               restore_best_weights = True)

  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                   factor = 0.2,
                                                   patience = 3,
                                                   verbose = 1,
                                                   min_delta = 0.00001)
  
  callbacks_list = [checkpoint, earlystop, reduce_lr]

 	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
  
  history = model.fit(train_data_generator, epochs=50, callbacks=callbacks_list,
                      validation_data=valid_data_generator, batch_size=batch_size)
  #PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
  
  model.load_weights("/content/model_"+str(fold_var)+".h5")
  
  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names,results))
  
  VALIDATION_ACCURACY.append(results['accuracy'])
  # VALIDATION_LOSS.append(results['loss'])
  
  tf.keras.backend.clear_session()
  fold_var += 1


In [23]:
VALIDATION_ACCURACY

[0.274954617023468,
 0.27767693996429443,
 0.2813067138195038,
 0.26248863339424133,
 0.25522252917289734]

In [24]:
from keras.models import load_model

classifier = load_model('/content/model_3.h5')

In [25]:
#preparing the train directory for keras
import tensorflow as tf

import pandas as pd
import shutil
import os
import sys
from tqdm import tqdm
import cv2
import numpy as np

test = pd.read_csv('/content/dataset/test.csv')
IMG_SIZE = 224

def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [26]:
test_img = []
for img_name in tqdm(test['image_id'].values):
    test_img.append(img_name + '.jpg')
    
test['image'] = test_img

test_img = []
for img_name in tqdm(test['image'].values): 
    test_img.append(read_img('/content/dataset/test/' + img_name))

100%|██████████| 2361/2361 [00:01<00:00, 1824.91it/s]


In [27]:
x_test = np.array(test_img, np.float32) / 255
predictions = classifier.predict(x_test)
y_classes = predictions.argmax(axis=-1)

In [28]:
label_map = (train_data_generator.class_indices)
rev_y = {v:k for k,v in label_map.items()}


In [29]:
output = pd.DataFrame({'image_id': test.image_id, 'breed': y_classes})
output['breed'] = output['breed'].map(rev_y)

In [30]:
filename = 'solution.csv'
output.to_csv(filename, index=False) 
output.head()

,image_id,breed
0,7ede553357,nerodia-sipedon
1,3500b219e4,storeria-dekayi
2,d43a78d6d9,nerodia-sipedon
3,1f36f26994,diadophis-punctatus
4,8cb85fc58e,pituophis-catenifer
